In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import traci
import sys
import argparse
import tensorflow as tf
import numpy as np
from dotenv import load_dotenv

# Import internal modules
from rl_package.rl_logic.Environnement import EnvironnementSumo
from rl_package.rl_logic.Agent import AgentSumo
from rl_package.params import *
print(BATCH_SIZE,NAME_SIMULATION,WINDOW)
print(SUMO_GUI_BIN,SUMO_BIN)
print(SIMUL_CONFIG)

✅ Simulation terminée avec mise à jour correcte des phases à chaque `WINDOW` step.
32 simu_opera_3 20
/Users/arseneclaustre/sumo/bin/sumo-gui /Users/arseneclaustre/sumo/bin/sumo
/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm2.sumocfg


In [3]:


# Load environment variables
#load_dotenv()


#SUMO_BIN = r"C:/Program Files/rl_project/Eclipse/Sumo/bin/sumo.exe"
# SIMUL_CONFIG = r"double_traffic/double_traffic.sumo.cfg"
#SIMUL_CONFIG = "/Users/arseneclaustre/code/psels/RL_traffic/Traffic/double_traffic/double_traffic.sumo.cfg"
#SIMUL_CONFIG = "/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.sumocfg"
# SIMUL_CONFIG = "/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera/osm.polycfg"
#BATCH_SIZE=6
# SUMO command

In [4]:
def preprocess():
    """
    Determines the number of inputs and outputs required for each agent.
    """
    sumoCmd = [SUMO_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings']
    print(sumoCmd)
    env = EnvironnementSumo(sumoCmd, WINDOW)
    inputs_per_agents = []
    outputs_per_agents = []
    positions_phases = []

    for trafficlight in env.trafficlights_ids:
        # Get the number of lanes controlled by this traffic light
        n_lanes = len(env.control_lanes(trafficlight))
        inputs_per_agents.append(n_lanes * 3)  # Inputs: queue + vehicle count

        # Get the number of valid traffic light phases (excluding yellow)
        n_phases,position = env.get_phase_without_yellow(trafficlight)
        #print(f'trafficlight :{trafficlight},\n lane associated {env.control_lanes(trafficlight)}')
        n_outputs = len(n_phases)
        positions_phases.append(position)
        outputs_per_agents.append(n_outputs)

    env.close()
    return inputs_per_agents, outputs_per_agents,positions_phases  # List of inputs, outputs per agent, and the postion phases of each trafficlight


In [5]:
def train_models(inputs_per_agents, outputs_per_agents, position_phases, type_model="DQN"):
    """
    Trains multiple reinforcement learning agents to optimize traffic lights.
    Saves each model separately.
    """
    agents = [AgentSumo(type_model, inputs, outputs) for inputs, outputs in zip(inputs_per_agents, outputs_per_agents)]

    # Load pre-trained models if available
    for i, agent in enumerate(agents):
        agent.build_model()
        model_path = f"models/{NAME_SIMULATION}_{type_model}_Agent{i}.keras"
        if os.path.exists(model_path):
            print(f"🔄 Loading pre-trained model for Agent {i} from {model_path}...")
            agent.model_action = tf.keras.models.load_model(model_path)
            if agent.model_target:  # For Double/Dueling DQN
                agent.model_target = tf.keras.models.load_model(model_path)

    sumoCmd = [SUMO_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings','--scale','0.6']



    for episode in range(EPISODE):
        print(f'🔄 Episode {episode}/{EPISODE}')
        env = EnvironnementSumo(sumoCmd, WINDOW)

        #Store the position phases of the trafficlight in the environment
        env.position_phases = position_phases

        epsilon = max(1 - episode / EPISODE, 0.01)  # Decaying epsilon for exploration

        traffic_lights = env.trafficlights_ids
        states = [env.get_states_per_traffic_light(traffic_light) for traffic_light in traffic_lights]

        for _ in range(75):  # Steps per episode
            actions = [agent.epsilon_greedy_policy(np.array(states[i]), epsilon) for i, agent in enumerate(agents)]
            next_states, rewards = env.step(actions)

            for i in range(len(agents)):
                agents[i].add_to_memory(np.array(states[i]), np.array(actions[i]), np.array(rewards[i]), np.array(next_states[i]))

            states = next_states

            if len(agents[0].replay_buffer) >= BATCH_SIZE *5:
                for agent in agents:
                    agent.training_step(BATCH_SIZE)

            if env.get_total_number_vehicles() == 0:
                break  # Stop simulation if no vehicles left

        if episode % 5 == 0 and type_model != 'DQN':
            for agent in agents:
                agent.model_target.set_weights(agent.model_action.get_weights())

        env.close()

    for i, agent in enumerate(agents):
        model_path = f"models/{NAME_SIMULATION}_{type_model}_Agent{i}.keras"
        agent.model_action.save(model_path)
        print(f"✅ Model saved for Agent {i} at: {model_path}")


In [6]:

def load_trained_agents(inputs_per_agents, outputs_per_agents, type_model="DQN"):
    """
    Loads pre-trained agents from saved model files.
    If any model is missing, exits the program.
    """
    agents = [AgentSumo(type_model, inputs, outputs) for inputs, outputs in zip(inputs_per_agents, outputs_per_agents)]

    for i, agent in enumerate(agents):
        model_path = f"models/{NAME_SIMULATION}_{type_model}_Agent{i}.keras"
        if os.path.exists(model_path):
            print(f"🔄 Loading pre-trained model for Agent {i} from {model_path}...")
            agent.build_model()
            agent.model_action = tf.keras.models.load_model(model_path)
        else:
            print(f"❌ No pre-trained model found for Agent {i}.")
            sys.exit(1)

    return agents

In [9]:
def scenario(agents,positions_phases):
    """
    Runs a SUMO simulation using the trained agents.
    """
    sumoCmd = [SUMO_GUI_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings','--scale','0.4']
    print(WINDOW)
    print(sumoCmd)
    env = EnvironnementSumo(sumoCmd, WINDOW)
    #Store the position phases of the trafficlight in the environment
    env.position_phases = positions_phases
    env.full_simul(agents)

In [8]:

type_model = "2DQN"
print(EPISODE,NAME_SIMULATION)
EPISODE = 30
inputs_per_agents, outputs_per_agents,positions_phases = preprocess()
print(f'inputs : {inputs_per_agents}')
print(f'outputs : {outputs_per_agents}')
print(f'positions : {positions_phases}')
train_models(inputs_per_agents, outputs_per_agents, positions_phases, type_model)
agents = load_trained_agents(inputs_per_agents, outputs_per_agents, type_model)
scenario(agents,positions_phases)


1 simu_opera_3
['/Users/arseneclaustre/sumo/bin/sumo', '-c', '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm2.sumocfg', '--start', '--no-warnings']
 Retrying in 1 seconds
***Starting server on port 60638 ***


Loading net-file from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.net.xml' ... done (24ms).
Loading additional-files from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.poly.xml.gz' ... done (3ms).
Loading route-files incrementally from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.passenger.trips.xml'
Loading done.
Simulation version v1_22_0+0748-755b84baf8f started with time: 0.00.
✅ All traffic light phases updated to fixed 30s durations.
Simulation ended at time: 0.00.
Reason: TraCI requested termination.
Performance:
 Duration: 0.02s
 TraCI-Duration: 0.00s
 Real time factor: 0
 UPS: 0.000000
Vehicles:
 Inserted: 0 (Loaded: 1)
 Running: 0
 Waiting: 0
Statistics (avg of 0):
 RouteLength: 0.00
 Speed: 0.00
 Duration: 0.00
 WaitingTime: 0.00
 TimeLoss: 0.00
 DepartDelay: 0.00
inputs : [12, 15, 15, 18, 9, 18, 3, 12, 15, 15, 30, 51, 36]
outputs : [2, 2, 2, 2, 2, 2, 1, 2, 3, 3, 4, 4, 5]
positions : [[0, 1], [0, 1], [0, 1], [0, 1],

/Users/arseneclaustre/code/psels/RL_traffic/rl_package/rl_logic/Environnement.py:25: UserWarning: Call to deprecated function getCompleteRedYellowGreenDefinition, use getAllProgramLogics instead.
  logic = traci.trafficlight.getCompleteRedYellowGreenDefinition(tl)[0]  # Récupère la logique actuelle
/Users/arseneclaustre/code/psels/RL_traffic/rl_package/rl_logic/Environnement.py:46: UserWarning: Call to deprecated function setCompleteRedYellowGreenDefinition, use setProgramLogic instead.
  traci.trafficlight.setCompleteRedYellowGreenDefinition(tl, new_logic)


🔄 Loading pre-trained model for Agent 3 from models/simu_opera_3_2DQN_Agent3.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 4 from models/simu_opera_3_2DQN_Agent4.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 5 from models/simu_opera_3_2DQN_Agent5.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 6 from models/simu_opera_3_2DQN_Agent6.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 7 from models/simu_opera_3_2DQN_Agent7.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 8 from models/simu_opera_3_2DQN_Agent8.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 9 from models/simu_opera_3_2DQN_Agent9.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 10 from models/simu_opera_3_2DQN_Agent10.keras...
🚀 Création d'un nouveau modèle 2DQN..

Loading net-file from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.net.xml' ... done (35ms).
Loading additional-files from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.poly.xml.gz' ... done (4ms).
Loading route-files incrementally from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.passenger.trips.xml'
Loading done.
Simulation version v1_22_0+0748-755b84baf8f started with time: 0.00.
✅ All traffic light phases updated to fixed 30s durations.
Simulation ended at time: 975.00.
Reason: TraCI requested termination.
Performance:
 Duration: 1.17s
 TraCI-Duration: 0.99s
 Real time factor: 832.622
 UPS: 40701.964133
Vehicles:
 Inserted: 300 (Loaded: 521)
 Running: 96
 Waiting: 2
 Teleports: 1 (Yield: 1)
Statistics (avg of 204):
 RouteLength: 543.91
 Speed: 4.15
 Duration: 154.36
 WaitingTime: 70.11
 TimeLoss: 87.63
 DepartDelay: 2.03
DijkstraRouter answered 307 queries and explored 65.12 edges on average.
DijkstraRouter spent 0.00s answ

...

Loading net-file from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.net.xml' ... done (34ms).
Loading additional-files from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.poly.xml.gz' ... done (4ms).
Loading route-files incrementally from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.passenger.trips.xml'
Loading done.
Simulation version v1_22_0+0748-755b84baf8f started with time: 0.00.
✅ All traffic light phases updated to fixed 30s durations.
Simulation ended at time: 975.00.
Reason: TraCI requested termination.
Performance:
 Duration: 65.71s
 TraCI-Duration: 64.90s
 Real time factor: 14.8377
 UPS: 578.487620
Vehicles:
 Inserted: 302 (Loaded: 521)
 Running: 52
 Waiting: 0
 Teleports: 3 (Jam: 2, Yield: 1)
 Emergency Stops: 1
 Emergency Braking: 1
Statistics (avg of 250):
 RouteLength: 563.16
 Speed: 5.08
 Duration: 129.50
 WaitingTime: 45.60
 TimeLoss: 60.47
 DepartDelay: 2.91
DijkstraRouter answered 309 queries and explored 64.21

Loading net-file from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.net.xml' ... done (39ms).
Loading additional-files from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.poly.xml.gz' ... done (5ms).
Loading route-files incrementally from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.passenger.trips.xml'
Loading done.
Simulation version v1_22_0+0748-755b84baf8f started with time: 0.00.
✅ All traffic light phases updated to fixed 30s durations.
Simulation ended at time: 975.00.
Reason: TraCI requested termination.
Performance:
 Duration: 67.00s
 TraCI-Duration: 66.22s
 Real time factor: 14.5529
 UPS: 709.733272
Vehicles:
 Inserted: 296 (Loaded: 521)
 Running: 81
 Waiting: 6
 Teleports: 11 (Jam: 2, Yield: 8, Wrong Lane: 1)
 Emergency Stops: 1
 Emergency Braking: 1
Statistics (avg of 215):
 RouteLength: 566.62
 Speed: 5.24
 Duration: 135.10
 WaitingTime: 50.83
 TimeLoss: 65.72
 DepartDelay: 1.90
DijkstraRouter answered 336 queries an

Loading net-file from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.net.xml' ... done (36ms).
Loading additional-files from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.poly.xml.gz' ... done (5ms).
Loading route-files incrementally from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.passenger.trips.xml'
Loading done.
Simulation version v1_22_0+0748-755b84baf8f started with time: 0.00.
✅ All traffic light phases updated to fixed 30s durations.
Simulation ended at time: 975.00.
Reason: TraCI requested termination.
Performance:
 Duration: 68.04s
 TraCI-Duration: 67.29s
 Real time factor: 14.3296
 UPS: 712.055966
Vehicles:
 Inserted: 301 (Loaded: 521)
 Running: 78
 Waiting: 1
 Teleports: 9 (Jam: 1, Yield: 8)
Statistics (avg of 223):
 RouteLength: 576.15
 Speed: 5.32
 Duration: 149.14
 WaitingTime: 63.60
 TimeLoss: 78.49
 DepartDelay: 3.51
DijkstraRouter answered 319 queries and explored 63.35 edges on average.
DijkstraRouter spent 0.

Loading net-file from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.net.xml' ... done (38ms).
Loading additional-files from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.poly.xml.gz' ... done (5ms).
Loading route-files incrementally from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.passenger.trips.xml'
Loading done.
Simulation version v1_22_0+0748-755b84baf8f started with time: 0.00.
✅ All traffic light phases updated to fixed 30s durations.
Simulation ended at time: 975.00.
Reason: TraCI requested termination.
Performance:
 Duration: 66.90s
 TraCI-Duration: 66.07s
 Real time factor: 14.5742
 UPS: 767.993543
Vehicles:
 Inserted: 301 (Loaded: 521)
 Running: 88
 Waiting: 1
 Teleports: 11 (Jam: 2, Yield: 9)
Statistics (avg of 213):
 RouteLength: 575.03
 Speed: 5.25
 Duration: 151.91
 WaitingTime: 65.74
 TimeLoss: 80.89
 DepartDelay: 1.84
DijkstraRouter answered 327 queries and explored 61.89 edges on average.
DijkstraRouter spent 0

FatalTraCIError: Connection closed by SUMO.

In [ ]:
print(NAME_SIMULATION)

simu_opera


In [16]:
type_model = "2DQN"
print(SIMUL_CONFIG)
#SIMUL_CONFIG = "/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Traditional_traffic/traditional_traffic.sumo.cfg"
EPISODE = 50
print(NAME_SIMULATION)
inputs_per_agents, outputs_per_agents,positions_phases = preprocess()

/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.sumocfg
simu_opera_3
['/Users/arseneclaustre/sumo/bin/sumo', '-c', '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.sumocfg', '--start', '--no-warnings']
 Retrying in 1 seconds
***Starting server on port 60416 ***


Loading net-file from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.net.xml.gz' ... done (38ms).
Loading additional-files from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.poly.xml.gz' ... done (4ms).
Loading route-files incrementally from '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm.passenger.trips.xml'
Loading done.
Simulation version v1_22_0+0748-755b84baf8f started with time: 0.00.
✅ All traffic light phases updated to fixed 30s durations.
Simulation ended at time: 0.00.
Reason: TraCI requested termination.
Performance:
 Duration: 0.03s
 TraCI-Duration: 0.00s
 Real time factor: 0
 UPS: 0.000000
Vehicles:
 Inserted: 0 (Loaded: 1)
 Running: 0
 Waiting: 0
Statistics (avg of 0):
 RouteLength: 0.00
 Speed: 0.00
 Duration: 0.00
 WaitingTime: 0.00
 TimeLoss: 0.00
 DepartDelay: 0.00


In [ ]:
print(NAME_SIMULATION)

simu_opera_2


In [10]:
agents = load_trained_agents(inputs_per_agents, outputs_per_agents, type_model)

🔄 Loading pre-trained model for Agent 0 from models/simu_opera_3_2DQN_Agent0.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 1 from models/simu_opera_3_2DQN_Agent1.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 2 from models/simu_opera_3_2DQN_Agent2.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 3 from models/simu_opera_3_2DQN_Agent3.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 4 from models/simu_opera_3_2DQN_Agent4.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 5 from models/simu_opera_3_2DQN_Agent5.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 6 from models/simu_opera_3_2DQN_Agent6.keras...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 7 from models/simu_opera_3_2DQN_Agent7.keras...
🚀 Création d'un nouveau modèle 2DQN...


In [11]:
scenario(agents,positions_phases)

20
['/Users/arseneclaustre/sumo/bin/sumo-gui', '-c', '/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Opera_2/osm2.sumocfg', '--start', '--no-warnings', '--scale', '0.4']
 Retrying in 1 seconds
✅ All traffic light phases updated to fixed 30s durations.
🔄 Step 0: Computing actions for traffic lights...
🚦 Traffic Light 25207334 - Applying Phase Index: 1
🚦 Traffic Light 25272923 - Applying Phase Index: 0
🚦 Traffic Light 29604212 - Applying Phase Index: 0
🚦 Traffic Light 398792 - Applying Phase Index: 0
🚦 Traffic Light GS_29604238 - Applying Phase Index: 0
🚦 Traffic Light cluster_10770535269_2200327356_2200327370_264494546 - Applying Phase Index: 0
🚦 Traffic Light cluster_11066733188_11066733189_11768662170_1757649347 - Applying Phase Index: 0
🚦 Traffic Light cluster_12529386650_12529386652_2200327352_25207337_#1more - Applying Phase Index: 0
🚦 Traffic Light cluster_2200327376_25273781_25273782_25273784 - Applying Phase Index: 2
🚦 Traffic Light cluster_25273876_25273887_6206597879 - Ap

FatalTraCIError: Connection closed by SUMO.